In [33]:
import cv2
import numpy as np
from pandas import DataFrame, read_csv
from matplotlib import pyplot as plt
from shutil import copy2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Dense, Activation, Input, Conv2D, MaxPooling2D, Flatten,
                                     GlobalMaxPooling2D, LeakyReLU, concatenate, Dropout)
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from os.path import isfile

In [34]:
SEED = 42

In [35]:
y_filename='training_solutions_rev1.csv'
y = read_csv(y_filename, index_col=0)[['Class1.1','Class1.2','Class1.3']]

In [36]:
crop_shape = (256,256)
input_shape = (96,96)

In [37]:
def create_model(input_shape=(input_shape[0],input_shape[1],3)):
    return Sequential([
        Input(shape=input_shape),
        
        Conv2D(8,(3,3), padding='same', input_shape=input_shape),
        Activation('relu'),
        MaxPooling2D(pool_size=(2,2)),
        
        Conv2D(16,(3,3), padding='same'),
        Activation('relu'),
        MaxPooling2D(pool_size=(2,2)),
        
        Conv2D(4,(3,3), padding='same'),
        Activation('relu'),
        MaxPooling2D(pool_size=(2,2)),
        
        Conv2D(4,(3,3), padding='same'),
        Activation('relu'),
        MaxPooling2D(pool_size=(2,2)),
        
        Flatten(),
        
        Dense(64),
        Activation('relu'),
        Dense(32),
        Activation('relu'),
        Dense(3),
        Activation('softmax')
])

In [38]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=input_shape,
        batch_size=50,
        class_mode='categorical',
        seed=SEED)

Found 43104 images belonging to 3 classes.


In [39]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

loss='categorical_crossentropy'
LEARNING_RATE = 0.05
BATCH_SIZE = 50
EPOCHS = 4
filename = './CNN-checkpoint-model.hdf5'
callback = ModelCheckpoint(filename, monitor='loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto', #save_freq=1000)
                             period=1)
if isfile(filename):
    model = load_model(filename, compile=False)
else:
    model = create_model()
model.compile(loss=loss, optimizer='adamax', metrics=[root_mean_squared_error])

In [40]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 96, 96, 8)         224       
_________________________________________________________________
activation_88 (Activation)   (None, 96, 96, 8)         0         
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 48, 48, 16)        1168      
_________________________________________________________________
activation_89 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 24, 24, 4)       

In [ ]:
with tf.device('/device:CPU:0'):
    train_generator.batch_size = BATCH_SIZE
    history = model.fit_generator(train_generator, epochs=EPOCHS,
                        verbose=1, callbacks=[callback])

Epoch 1/4
837/863 [============================>.] - ETA: 3s - loss: 5.5633 - root_mean_squared_error: 0.4777

In [ ]:
DataFrame(history.history).plot(figsize=(8, 5))#, logy=True)
plt.grid(True)
#plt.gca().set_ylim(0, 1)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
valid_datagen = ImageDataGenerator()
valid_generator = valid_datagen.flow_from_directory(
        './test',
        target_size=input_shape,
        batch_size=50,
        class_mode='categorical',
        seed=SEED)

In [ ]:
valid_generator.class_indices

In [ ]:
with tf.device('/device:CPU:0'):
    y_pred = model.predict_generator(valid_generator)
y_pred.shape

In [ ]:
valid_generator.batch_size = valid_generator.samples
X_valid, y_valid = valid_generator.next()

In [ ]:
y_valid.shape

In [ ]:
y_pred.argmax(axis=1)

In [ ]:
#y_train_class = y_batch.argmax(axis=1)
#y_pred_class = y_pred.argmax(axis=1)
conf_mat = confusion_matrix(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
plt.figure(figsize=(15,10))
sns.heatmap(conf_mat,square=True, annot=True, fmt='.0f')
plt.show()
print('Accuracy=', accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
y_train_pred = model.predict_generator(train_generator)

In [ ]:
train_generator.batch_size = train_generator.samples
y_train = train_generator.next()[1]

In [ ]:
y_train_class = y_train.argmax(axis=1)
y_train_pred_class = y_train_pred.argmax(axis=1)
conf_mat = confusion_matrix(y_train_class, y_train_pred_class)
plt.figure(figsize=(15,10))
sns.heatmap(conf_mat,square=True, annot=True, fmt='.0f')
plt.show()
print('Accuracy=', accuracy_score(y_train_class, y_train_pred_class))